In [2]:
################################################
## setup to use GPU TensorFlow on Colab -- cwr
################################################

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()

#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

if device_name == '/device:GPU:0':
  print('Found GPU at: {}'.format(device_name))
else:
  print('GPU device not found')

Found GPU at: /device:GPU:0


In [3]:
import PIL
import numpy as np
from PIL import Image
from numpy import asarray
import matplotlib.pyplot as plt

# from DLAVA, includes unused symbols, maybe tighten later
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

# trying to get around memory limits:
from random import sample

In [4]:
# Use the "mini" training set with 9 images.
# fcd_ts_dir = '/content/drive/My Drive/PredatorEye/mini_training_set/'

# Use the "real" training set with 2008 images
fcd_ts_dir = '/content/drive/My Drive/PredatorEye/fcd_training_set/'


In [5]:
# Parse FCD filename to a list of (x, y) pixel coordinates.
def fcd_filename_to_xy_pair(filename):
    without_extension = filename.split('.')[0]
    two_numeric_strings = without_extension.split('_')[1:3]
    two_ints = list(map(int, two_numeric_strings))
    return two_ints

In [6]:
# Prototype function to load FCD data from directory pathname

fcd_images = []
fcd_labels = []

# Maybe read from image file?
# Maybe assert all images are square and this size?
fcd_image_size = 1024

from os import listdir
from os.path import join

def fcd_normalized_xy(filename, pixels):
    pixel_coordinates = fcd_filename_to_xy_pair(filename)
    normalized = [pixel_coordinates[0] / pixels.shape[0],
                  pixel_coordinates[1] / pixels.shape[1]]
    return normalized

# def read_fcd_data_from_directory(directory_pathname):
#     for filename in listdir(directory_pathname):
#         # Numpy pixel array of image object.
#         pixels = asarray(Image.open(join(directory_pathname, filename)))
#         # Accumulate: numpy pixel array of image object.
#         fcd_images.append(pixels)
#         # Accumulate: disk center XY position ground truth.
#         fcd_labels.append(np.array(fcd_normalized_xy(filename, pixels)))
#         # Progress report (runs for about 6 minutes on 20211127)
#         if ((len(fcd_images) % 100) == 0):
#              print(len(fcd_images), "images...")



def read_fcd_data_from_directory(directory_pathname):
    directory_contents = listdir(directory_pathname)
    #
    # trying to get around memory limits:
    directory_contents = sample(directory_contents, 100)
    #
    for filename in directory_contents:
        # Numpy pixel array of image object.
        pixels = asarray(Image.open(join(directory_pathname, filename)))
        # Accumulate: numpy pixel array of image object.
        fcd_images.append(pixels)
        # Accumulate: disk center XY position ground truth.
        fcd_labels.append(np.array(fcd_normalized_xy(filename, pixels)))
        # Progress report (runs for about 6 minutes on 20211127)
        if ((len(fcd_images) % 100) == 0):
             print(len(fcd_images), "images...")

In [7]:
# Construct a striding CNN Keras model (cribbed from DLAVA chapter B3, Listing
# B3-41): currently 2 Conv2D with dropout, then 2 dense, outpu layer is dense
# with just two neurons, to generate an XY position estimate for the disk.

def make_striding_cnn_model():
    model = Sequential()
    model.add(Conv2D(30, (5, 5), activation='relu', padding='same',
                     strides=(2, 2), kernel_constraint=MaxNorm(3),
                     # input_shape=(image_height, image_width, 1)))
                     input_shape=(fcd_image_size, fcd_image_size, 3)))
    model.add(Dropout(0.2))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', 
                     strides=(2, 2), kernel_constraint=MaxNorm(3)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    #
    # instead of softmax, maybe a Dense layer with two neurons (x, y)?
    #
    # model.add(Dense(number_of_classes, activation='softmax'))
    model.add(Dense(2, activation='relu')) ### COMPLETELY UNTESTED!! ###
    #
    # Use MSE in place of categorical_crossentropy
    # since my problem is more like regression
    #
    # model.compile(loss='categorical_crossentropy', optimizer='adam', 
    #               metrics=["accuracy"])
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer='adam',
                  metrics=["accuracy"])
    return model

In [8]:
# Utility to fit and plot a run, again cribbed from DLAVA chapter B3.

random_seed = 42

def run_model(model_maker, plot_title):
    model = model_maker()
    np.random.seed(random_seed)
    history = model.fit(X_train, y_train,
                        validation_split=0.25,
                        # epochs=100, batch_size=256) ####################
                        epochs=100, batch_size=10)
    plot_accuracy_and_loss(history, plot_title)
    return history

# A little utility to draw plots of accuracy and loss.
def plot_accuracy_and_loss(history, plot_title):
    xs = range(len(history.history['accuracy']))

    plt.figure(figsize=(10,3))
    plt.subplot(1, 2, 1)
    plt.plot(xs, history.history['accuracy'], label='train')
    plt.plot(xs, history.history['val_accuracy'], label='validation')
    plt.legend(loc='lower left')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.title(plot_title+', Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(xs, history.history['loss'], label='train')
    plt.plot(xs, history.history['val_loss'], label='validation')
    plt.legend(loc='upper left')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.title(plot_title+', Loss')

    plt.show()

In [9]:
# Read FCD training data from a given directory.
print('Reading all images from ' + fcd_ts_dir)
print('...')
read_fcd_data_from_directory(fcd_ts_dir)
print('Read', len(fcd_images), 'training images.')

Reading all images from /content/drive/My Drive/PredatorEye/fcd_training_set/
...
100 images...
Read 100 training images.


In [10]:
# Run a model.
X_train = np.array(fcd_images)
y_train = np.array(fcd_labels)

print(type(X_train))
print(X_train.shape)
print(y_train.shape)

striding_history = run_model(make_striding_cnn_model, 'FCD: Striding CNN')

<class 'numpy.ndarray'>
(100, 1024, 1024, 3)
(100, 2)
Epoch 1/100
8/8 [==============================] - 14s 474ms/step - loss: 3.5960 - accuracy: 0.5467 - val_loss: 0.3327 - val_accuracy: 0.4400
Epoch 2/100
8/8 [==============================] - 2s 302ms/step - loss: 0.3211 - accuracy: 0.5467 - val_loss: 0.3327 - val_accuracy: 0.4400
Epoch 3/100
8/8 [==============================] - 2s 302ms/step - loss: 0.3211 - accuracy: 0.5467 - val_loss: 0.3327 - val_accuracy: 0.4400
Epoch 4/100
8/8 [==============================] - 2s 305ms/step - loss: 0.3211 - accuracy: 0.5467 - val_loss: 0.3327 - val_accuracy: 0.4400
Epoch 5/100
8/8 [==============================] - 2s 302ms/step - loss: 0.3211 - accuracy: 0.5467 - val_loss: 0.3327 - val_accuracy: 0.4400
Epoch 6/100
8/8 [==============================] - 2s 304ms/step - loss: 0.3211 - accuracy: 0.5467 - val_loss: 0.3327 - val_accuracy: 0.4400
Epoch 7/100
8/8 [==============================] - 2s 303ms/step - loss: 0.3211 - accuracy: 0.5467 

KeyboardInterrupt: ignored